In [1]:
import tensorflow as tf
print(tf.__version__)
from keras.utils import np_utils
import matplotlib.pyplot as plt

2.1.0


Using TensorFlow backend.


In [2]:


fashion_mnist = tf.keras.datasets.fashion_mnist

# load the training and test data    
(tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()

# reshape the feature data
tr_x = tr_x.reshape(tr_x.shape[0], 784)
te_x = te_x.reshape(te_x.shape[0], 784)

# noramlise feature data
tr_x = tr_x / 255.0
te_x = te_x / 255.0

print( "Shape of training features ", tr_x.shape)
print( "Shape of test features ", te_x.shape)


# one hot encode the training labels and get the transpose
tr_y = np_utils.to_categorical(tr_y,10)
tr_y = tr_y.T
print ("Shape of training labels ", tr_y.shape)

# one hot encode the test labels and get the transpose
te_y = np_utils.to_categorical(te_y,10)
te_y = te_y.T
print ("Shape of testing labels ", te_y.shape)


Shape of training features  (60000, 784)
Shape of test features  (10000, 784)
Shape of training labels  (10, 60000)
Shape of testing labels  (10, 10000)


In [3]:
#Reshaping Training data to n x m matrix 
tr_x_reshaped = tr_x.T 
#Reshaping test data to n x m matrix 
te_x_reshaped = te_x.T
print( "Shape of training features ", tr_x_reshaped.shape)
print( "Shape of test features ", te_x_reshaped.shape)
tr_y = tf.constant(tr_y)
te_y = tf.constant(te_y)
tr_x_reshaped = tf.constant(tr_x_reshaped)
te_x_reshaped = tf.constant(te_x_reshaped)

Shape of training features  (784, 60000)
Shape of test features  (784, 10000)


In [4]:
W = tf.Variable(tf.random.normal([ 10,tr_x_reshaped.shape[0]], mean=0.0, stddev=0.05))
b = tf.Variable(tf.zeros(shape = (10,1)))

print("Shape of Weights ", W.shape)

print("Shape of bias ", b.shape)



def forward_pass(X,W,b):
    WX = tf.matmul(tf.cast(W, tf.float32),tf.cast(X, tf.float32))
    A1 = tf.add(WX,b)
    H = softmax(A1)
    H = tf.clip_by_value(H, 1e-10, 1.0)
    return H

def cross_entropy(H,y):
    loss  = -tf.reduce_sum(tf.math.multiply(y, tf.math.log(H)),0)
    total_loss = tf.reduce_mean(loss)
    return total_loss
    
    
def calculate_accuracy(x,y,w,b):
    y_pred_softmax = forward_pass(x, w, b)
    predictions = tf.math.argmax(y_pred_softmax)
    actual_y = tf.math.argmax(y)
    predictions_correct = tf.cast(tf.equal(predictions, actual_y), tf.float32)
    accuracy = tf.reduce_mean(predictions_correct)
    return accuracy

def softmax(A):
    t = tf.math.exp(A)
    sum_of_t = tf.reduce_sum(t, 0)
    soft_max_output = tf.math.divide(t, sum_of_t)
    return soft_max_output



Shape of Weights  (10, 784)
Shape of bias  (10, 1)


In [ ]:
num_Iterations = 11000
adam_optimizer = tf.keras.optimizers.Adam()
trainingLoss= []
validationLoss= []
trainingAccuracies = []
validationAccuracies = []

for i in range(num_Iterations):
    with tf.GradientTape() as tape:
        y_pred = forward_pass(tr_x_reshaped, W, b)
        currentLoss = cross_entropy(y_pred,tr_y)
        gradients = tape.gradient(currentLoss, [W, b])
    accuracy = calculate_accuracy(tr_x_reshaped, tr_y, W, b)
    trainingLoss.append(currentLoss)
    trainingAccuracies.append(accuracy)
    y_pred_test = forward_pass(te_x_reshaped, W, b)
    currentLoss_test = cross_entropy(y_pred_test,te_y)
    accuracy_test = calculate_accuracy(te_x_reshaped, te_y, W, b)
    validationLoss.append(currentLoss_test)
    validationAccuracies.append(accuracy_test)
    #print ("Iteration ", i, ": Loss = ",currentLoss.numpy(), "  Acc: ", accuracy.numpy())
    #print ("")
    adam_optimizer.apply_gradients(zip(gradients, [W,b]))
print("Final Accuracy: ", accuracy.numpy() , "Final Loss: ", currentLoss.numpy())
print("Final Test Accuracy: ", accuracy_test.numpy() , "Final Test Loss: ", currentLoss_test.numpy())


In [ ]:
plt.plot(validationLoss, label="Val Loss")
plt.plot(validationAccuracies, label="Val Acc")

plt.plot(trainingLoss, label="Train Loss")
plt.plot(trainingAccuracies, label="Train Acc")
plt.legend()

plt.show()